# State Module Verification

This notebook tests the `src.state` module which defines:
- **GraphState TypedDict** - The state object that travels through LangGraph
- **add_messages reducer** - Ensures conversation history accumulates

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from src.state import GraphState
from langchain_core.messages import HumanMessage, AIMessage

## Step 1: Verify GraphState Structure

Check that all required keys are present.

In [ ]:
expected_keys = [
    "messages", "question", "standalone_question", "documents",
    "generation", "documents_relevant", "is_hallucination", "retry_count"
]

actual_keys = list(GraphState.__annotations__.keys())

print(f"Structure Check:")
print(f"   • Expected: {len(expected_keys)} keys")
print(f"   • Actual: {len(actual_keys)} keys")

missing = set(expected_keys) - set(actual_keys)
extra = set(actual_keys) - set(expected_keys)

if missing:
    print(f"   Missing: {missing}")
if extra:
    print(f"   Extra: {extra}")
if not missing:
    print("   All required keys present!")

## Step 2: Verify Type Annotations

In [ ]:
print("Type Annotations:")
for key, type_hint in GraphState.__annotations__.items():
    # Simplify display
    type_str = str(type_hint)
    if "Annotated" in type_str:
        type_str = "Annotated[List[BaseMessage], add_messages]"
    elif "typing.List" in type_str:
        type_str = "List[Any]"
    else:
        type_str = type_hint.__name__ if hasattr(type_hint, '__name__') else str(type_hint)
    print(f"   • {key}: {type_str}")

print("\nType annotations verified!")

## Step 3: Test State Creation

In [ ]:
state: GraphState = {
    "messages": [HumanMessage(content="Who is the CEO of Apple?")],
    "question": "Who is the CEO of Apple?",
    "standalone_question": "",
    "documents": [],
    "generation": "",
    "documents_relevant": "no",
    "is_hallucination": "no",
    "retry_count": 0
}

print(f"State Created:")
print(f"   • Keys: {len(state)}")
print(f"   • Messages: {len(state['messages'])}")
print(f"   • Question: '{state['question']}'")
print(f"   • Retry count: {state['retry_count']}")

print("\nState creation works!")

## Step 4: Test Messages Accumulation

Verify that messages accumulate across turns (this is what `add_messages` enables).

In [ ]:
# Simulate multi-turn conversation
messages = [
    HumanMessage(content="Who is the CEO of Apple?"),
    AIMessage(content="Tim Cook is the CEO of Apple."),
    HumanMessage(content="How old is he?"),
    AIMessage(content="Tim Cook was born in 1960.")
]

print(f"Multi-turn Conversation:")
for i, msg in enumerate(messages):
    role = "👤 Human" if isinstance(msg, HumanMessage) else "AI"
    print(f"   {role}: {msg.content}")

print(f"\n   Total messages: {len(messages)}")
print("\nMessages accumulate correctly!")